In [109]:
import numpy as np
from datetime import datetime
from collections import namedtuple
from quantfreedom.utils import pretty_qf
from quantfreedom.strategies.strategy import Strategy

from env import MufexKeys
from strat import RSIRisingFalling
from quantfreedom.enums import CandleBodyType

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest

from quantfreedom.exchanges.mufex import Mufex
from quantfreedom.helper_funcs import cart_product
from quantfreedom.helper_funcs import dos_cart_product

mufex_main = Mufex(
    api_key=MufexKeys.mainnet_api_key,
    secret_key=MufexKeys.mainnet_secret_key,
    use_test_net=False,
)
symbol = 'BTCUSDT'

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
A = namedtuple("A", "a b c")
B = namedtuple("B", "d e")
a = A(np.arange(20), 20, 30)
b = B(40, 50)
C = namedtuple("C", A._fields + B._fields)
C(*(a + b))

C(a=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19]), b=20, c=30, d=40, e=50)

In [111]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=3000,
)

In [112]:
dos_tuple = DynamicOrderSettings(
    max_trades=np.array([4]),
    account_pct_risk_per_trade=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.1]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

In [113]:
long_strat = RSIRisingFalling(
    long_short="long",
    rsi_length=np.array([14]),
    rsi_is_below=np.array([40, 60, 80]),
)
long_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=0,
)
long_strat.plot_signals(candles=candles)

In [114]:
pretty_qf(long_strat.indicator_settings_tuple)

IndicatorSettings(
    rsi_is_above = [0],
    rsi_is_below = [40 60 80],
    rsi_length = [14],
)


In [115]:
pretty_qf(dos_tuple)

DynamicOrderSettings(
    max_trades = [4],
    account_pct_risk_per_trade = [3],
    risk_reward = [2 5],
    sl_based_on_add_pct = [0.10],
    sl_based_on_lookback = [20],
    sl_bcb_type = [3],
    sl_to_be_cb_type = [6],
    sl_to_be_when_pct = [0],
    trail_sl_bcb_type = [3],
    trail_sl_by_pct = [0.50 1.00],
    trail_sl_when_pct = [1 2],
)


In [116]:
A = namedtuple("A", "a b c")
B = namedtuple("B", "d e")
a = A(np.arange(20), 20, 30)
b = B(40, 50)
C = namedtuple("C", A._fields + B._fields)
C(*(a + b))
comb = namedtuple("combined", dos_tuple._fields + long_strat.indicator_settings_tuple._fields)
new_comb = comb(*(dos_tuple + long_strat.indicator_settings_tuple))
pretty_qf(new_comb)

combined(
    max_trades = [4],
    account_pct_risk_per_trade = [3],
    risk_reward = [2 5],
    sl_based_on_add_pct = [0.10],
    sl_based_on_lookback = [20],
    sl_bcb_type = [3],
    sl_to_be_cb_type = [6],
    sl_to_be_when_pct = [0],
    trail_sl_bcb_type = [3],
    trail_sl_by_pct = [0.50 1.00],
    trail_sl_when_pct = [1 2],
    rsi_is_above = [0],
    rsi_is_below = [40 60 80],
    rsi_length = [14],
)


In [119]:
pretty_qf(dos_cart_product(dos_tuple=dos_tuple, strategy=long_strat))

DynamicOrderSettings(
    max_trades = [4 4 4 ... 4 4 4],
    account_pct_risk_per_trade = [0.03 0.03 0.03 ... 0.03 0.03 0.03],
    risk_reward = [2.00 2.00 2.00 ... 5.00 5.00 5.00],
    sl_based_on_add_pct = [0.00 0.00 0.00 ... 0.00 0.00 0.00],
    sl_based_on_lookback = [20 20 20 ... 20 20 20],
    sl_bcb_type = [3 3 3 ... 3 3 3],
    sl_to_be_cb_type = [6 6 6 ... 6 6 6],
    sl_to_be_when_pct = [0.00 0.00 0.00 ... 0.00 0.00 0.00],
    trail_sl_bcb_type = [3 3 3 ... 3 3 3],
    trail_sl_by_pct = [0.01 0.01 0.01 ... 0.01 0.01 0.01],
    trail_sl_when_pct = [0.01 0.01 0.01 ... 0.02 0.02 0.02],
)


In [118]:
pretty_qf(long_strat.indicator_settings_tuple)

IndicatorSettings(
    rsi_is_above = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],
    rsi_is_below = [40 60 80 40 60 80 40 60 80 40 60 80 40 60 80 40 60 80 40 60 80 40 60 80],
    rsi_length = [14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14],
)
